# Transcriptomics Tutorials
This series of notebooks is created to showcase transcript analysis on files. The series consists of the following notebooks:
- Notebook 1: Expression Data Transformation
- Notebook 2: Differential Expression Analysis
- Notebook 3: Gene Set Enrichment Analysis
- Notebook 4: Gene Co-Expression Analysis
- Notebook 5: Gene Regulatory Network

# Notebook 4: Gene Co-Expression Analysis

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

<a href="https://github.com/dnanexus/OpenBio/blob/master/LICENSE.md">MIT License</a> applies to this notebook.

In this notebook, we will use weighted gene co-expression network analysis (WGCNA) to identify which genes are co-expressed in our samples. WGCNA uses a series of correlations to identify sets of genes that are expressed together in our data.

## 1. Preparing your environment

<b>Launch spec:</b> 
- App name: JupyterLab with Python, R, Stata, ML
- Kernel: R
- Instance type: mem1_ssd1_v2_x16
- cost: < $0.25
- runtime: =~ 15 min


<b>Data description:</b> File input for this notebook is 
1. A matrix of samples and their respective gene expression counts.This file has the expression counts of 60,483 genes for 60 samples (30 normal, 30 tumor).
2. A summary file giving the file names and IDs of normal tissue and tumor samples.

<b>Package dependency:</b> 

| Package | License | 
| --- | --- |
| tidyverse | <a href="https://github.com/dnanexus/OpenBio/blob/master/LICENSE.md">MIT License</a> + <a href="https://cran.r-project.org/web/packages/tidyverse/LICENSE">file LICENSE</a> |
| org.Hs.eg.db | <a href="https://opensource.org/licenses/Artistic-2.0">artistic-2.0 </a> |
| WGCNA | <a href="https://cran.r-project.org/web/licenses/GPL-2">GPL-2 </a>, <a href="https://cran.r-project.org/web/licenses/GPL-3">GPL-3 </a> |
| topGO | <a href="https://www.gnu.org/licenses/lgpl-3.0.en.html">LGPL </a> |

**Install Packages**

Uncomment the install commands if you are comfortable with the library license and want to install and run the parts notebook that depend on the library.

_Note: Package installation takes ~10 minutes_

In [ ]:
# Install the libraries impute, WGCNA, org.Hs.eg.db, topGO from Bioconductor
# BiocManager::install("impute")        # A dependency of WGCNA that needs to be installed before installing WGCNA
# BiocManager::install("WGCNA")         # Gene co-expression analysis package
# BiocManager::install("org.Hs.eg.db")  # This is our human-specific annotation package
# BiocManager::install("topGO")         # Gene Ontology package
# Install the library tidyverse from CRAN
# install.packages("tidyverse")         # Required for data handling

**Declare input and output file names**

In notebook 1: Expression Data Transformation, we generated a counts matrix file from individual gene expression files (CPTAC-3_gene_expression_count_matrix.csv), a pheontype summary table (CPTAC-3_pheno_summary.csv), and saved them in our project on the DNAnexus platform. Select the files to be downloaded and the filename of the output files of this notebook.

In [ ]:
# Input files
counts_file <- "CPTAC-3_gene_expression_count_matrix.csv"
pheno_file <- "CPTAC-3_pheno_summary.csv"

# Output file
go_file <- "CPTAC-3_top_GO_terms_all_modules.csv"

**Download Data**

We download these files using CLI dx-toolbox command, `dx download <file_name>`.

In [ ]:
# Download files from DNAnexus platform
system(paste("dx download", counts_file))
system(paste("dx download", pheno_file))

_Note: At this point, we suggest creating a snapshot of the environment for resuse --> DNAnexus/Create Snapshot. Once a snapshot is created, the object may be used when launching a new JupyterLab instance and will contain all installed packages and any downloaded data._

## 2. Load Libraries

In [ ]:
library(WGCNA)
library(org.Hs.eg.db)
library(topGO)
library(tidyverse)

## 3. Load Data

In [11]:
# Read in sample expression counts
counts_df <- read_csv(counts_file, show_col_types = FALSE)
colnames(counts_df)[1:5]
dim(counts_df)

# Read in sample phenotypes
pheno_df <- read_csv(pheno_file, show_col_types = FALSE)
colnames(pheno_df)
dim(pheno_df)

[1] "gene"                                
[2] "0b5a5d3f-0716-4b0c-97eb-773fda630c4a"
[3] "0b5bb378-3634-476b-90a5-12fce87d85cc"
[4] "0df7d07d-67cd-4002-9b2b-771c5eb0d128"
[5] "1139922f-b5e7-49cf-b111-524a79419c84"

[1] 60483    61

[1] "case_ids"                   "normal_file_ids"           
 [3] "normal_file_names"          "normal_created_date"       
 [5] "normal_updated_date"        "primary_tumor_file_ids"    
 [7] "primary_tumor_file_names"   "primary_tumor_created_date"
 [9] "primary_tumor_updated_date" "date_of_download"

[1] 30 10

## 4. Subset, annotate, and transform source data

In [ ]:
# Set seed for repeatable randomization 
set.seed(200)

# Find all transcripts which contain any instance of the value, 0
genes <- column_to_rownames(counts_df, var = "gene") %>%
    apply(1, FUN = min) %>%
    stack() %>%
    rename(ensembl_id = ind) %>%
    filter(values > 0)

# Get all GO annotations with corresponding ENSG value
allGO2genes <- annFUN.org(
    whichOnto="BP",
    feasibleGenes=NULL,
    mapping="org.Hs.eg.db",
    ID="ensembl")

# Create a unique list of all GO annotated ENSG genes (~21K genes)
go_annotated_ensembl_ids <- stack(allGO2genes) %>%
    rename(ensembl_ids = values) %>%
    select(ensembl_ids) %>%
    distinct() %>%
    pull(ensembl_ids)

# Subset counts table by removing transcripts containing 0,
# subset by GO annoated genes, and randomly sample a subset of transcripts
counts_df <- counts_df %>%
    filter(gene %in% genes$ensembl_id) %>%
    separate(gene, c("gene", NA)) %>%
    filter(gene %in% go_annotated_ensembl_ids) %>%
    sample_n(1000) %>%
    column_to_rownames("gene") %>%
    t()
    
# Show preview of resulting matrix
head(counts_df, 3)
dim(counts_df)

### Phenotype Data Transformation

In [ ]:
tum <- pheno_df %>%
    select(case_ids, primary_tumor_file_ids) %>%
    rename(case_id = case_ids, sample_id = primary_tumor_file_ids) %>%
    mutate(sample_type = "tumor") %>%
    select(case_id, sample_id, sample_type)

nor <- pheno_df %>%
    select(case_ids, normal_file_ids) %>%
    rename(case_id = case_ids, sample_id = normal_file_ids) %>%
    mutate(sample_type = "normal") %>%
    select(case_id, sample_id, sample_type)

pheno_df <- tum %>% bind_rows(nor)
head(pheno_df, 3)

## 5. Show heatmap of transcript co-expression for Tumor and Normal sample sets

### Transcript co-expression heatmap for Tumor samples

In [ ]:
tumor_samples <- pheno_df %>%
    filter(sample_type == "tumor") %>%
    pull(sample_id)

cm = cor(counts_df[tumor_samples,])
heatmap(cm, labRow = FALSE, labCol = FALSE, main = "Gene co-expression: Tumor samples")

### Transcript co-expression heatmap for Normal samples

In [ ]:
normal_samples <- pheno_df %>%
    filter(sample_type == "normal") %>%
    pull(sample_id)

cm = cor(counts_df[normal_samples,])
heatmap(cm, labRow = FALSE, labCol = FALSE, main = "Gene co-expression: Normal samples")

## 6. WGCNA
<a href="https://horvath.genetics.ucla.edu/html/CoexpressionNetwork/Rpackages/WGCNA/#WGCNAIntro">WGCNA</a> is an R package for weighted correlation network analysis. For additional details in constructing a gene network, please see this public
 <a href="https://horvath.genetics.ucla.edu/html/CoexpressionNetwork/Rpackages/WGCNA/Tutorials/index.html">WGCNA tutorial</a>.

### Initiate and validate the structure of our data

In [ ]:
# Form multi-set expression data
multiexpr = vector(mode = "list", length = 2)
multiexpr[[1]] = list(data = as.data.frame(counts_df[tumor_samples, ]))
multiexpr[[2]] = list(data = as.data.frame(counts_df[normal_samples, ]))
print(checkSets(multiexpr))

### Construct Network

In [ ]:
# Set correlation function to WGCNA
cor <- WGCNA::cor
# Parallelize run and leverage multiple threads in mem1_ssd1_x2_16 instance
allowWGCNAThreads()
ALLOW_WGCNA_THREADS=16

# Build network
net <- blockwiseConsensusModules(
    multiExpr = multiexpr,
    power = 6,
    minModuleSize = 30,
    deepSplit = 2,
    pamRespectsDendro = FALSE,
    mergeCutHeight = 0.25,
    numericLabels = TRUE,
    minKMEtoStay = 0,
    saveTOMs = FALSE,
    verbose = 1)

In [ ]:
# Report module count and module size
table(net$colors)

### Plot hierarchical clustering dendrogram (tree) of our modules

In [ ]:
plotDendroAndColors(
    dendro = net$dendrograms[[1]],
    colors = net$colors,
    groupLabels = "Modules",
    dendroLabels = FALSE,
    hang = 0.03,
    addGuide = TRUE,
    guideHang = 0.05,
    main = "Consensus gene dendrogram and modules by color")

### Compare eigengene networks in Tumor vs. Normal samples

In [ ]:
consmesc = multiSetMEs(multiexpr, universalColors = net$colors)
met = consensusOrderMEs(consmesc)
plotEigengeneNetworks(met,
                      setLabels = c("tumor", "normal"),
                      marDendro = c(0,2,2,1),
                      marHeatmap = c(3,3,2,1),
                      xLabelsAngle = 90)

## 7. Enrichment analysis (GO)

### Convert module list to DataFrame

In [ ]:
modules <- stack(net$colors) %>%
    rename(module = values, ensembl = ind)
head(modules, 3)
dim(modules)

### Perform enrichment analysis for each module and return top 10 results for each module

In [ ]:
# Create list of modules to iterate across
module_list <- modules %>%
    pull(module) %>%
    unique() %>%
    sort()

# Inititate a tibble to append all results
all_module_results <- tibble()

# Iterate of list of modules
for(m in module_list){
    print(paste("...", "processing module", m, "...",sep = " "))
    # Generate background genes
    background_genes <- modules %>% pull(ensembl)
    # Get all genes in for a specificed module
    module_set <- modules %>% filter(module == m) %>% pull(ensembl)
    gene_list_in_module <- factor(as.integer(background_genes %in% module_set))
    names(gene_list_in_module) = background_genes

    # Initiate new topGOdata class for a given module
    go_data <- new(
        "topGOdata",
        ontology = "BP",
        allGenes = gene_list_in_module,
        annot = annFUN.org,
        mapping = "org.Hs.eg.db",
        ID = "ensembl",
        nodeSize = 10)
    
    # Test for GO enrichment using genes from a given module
    module_results <- runTest(go_data, algorithm = "classic", statistic = "fisher")
    
    # Create dataframe for results, for a given module and add module
    module_go_enrichment <- GenTable(go_data, Fisher_pvalue = module_results, topNodes = 10) %>%
        mutate(module = m)
    all_module_results <- all_module_results %>%
        bind_rows(module_go_enrichment)
}

### Show results

In [ ]:
all_module_results

## 8. Upload module data to project

In [ ]:
# Export the dataframe containing the 10 best terms for each module and save it to our project
write_csv(all_module_results, file = go_file)
system(paste("dx upload", go_file))